In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# params
test = True
create_orders = False
fee = 0.00075
gain_threshold = 1.03
coinmarketcap_threshold = 100
arbitrage_asset = 'BNB'
initial_arbitrage_balance = 1.0

In [ ]:
from binance_utils import init, init_test, get_all_symbols, get_trunc_value, get_asset_balance, create_market_order
from marketcap_utils import get_coinmarketcap_data
import numpy as np
import pandas as pd

fee_factor = 1.0 - fee

if test:
    client = init_test()
    fee_factor = 1.0
else:
    client = init()

symbols_df = get_all_symbols(client)
coinmarketcap_df = get_coinmarketcap_data(num_pages=5)
symbols_df = symbols_df.join(coinmarketcap_df[['cmcRank']], on='baseAsset')
symbols_df.dropna(inplace=True)
symbols_df = symbols_df[symbols_df['cmcRank'] <= coinmarketcap_threshold]

free_balance_arbitrage, _ = get_asset_balance(client, arbitrage_asset)
if initial_arbitrage_balance > free_balance_arbitrage:
    initial_arbitrage_balance = free_balance_arbitrage

In [ ]:
info = client.get_account()
assets_before_arbitrage_df = pd.DataFrame(info['balances'])

In [ ]:
#to BUY: quantity of quote_asset.
#to SELL: quantity of base_asset.
def execute_arbitrage_orders(base_asset, quote_asset, side, qty):
    print('Creating orders...')
    symbol = [np.nan] * 3
    free_balance = 0.0

    symbol[0] = base_asset[0] + quote_asset[0]
    order0 = create_market_order(client, symbol[0], side[0], qty)
    print(order0)

    symbol[1] = base_asset[1] + quote_asset[1]
    if side[1] == 'SELL':
        qty = float(order0['executedQty']) * fee_factor
        free_balance, _ = get_asset_balance(client, base_asset[1])
    elif side[1] == 'BUY':
        qty = float(order0['cummulativeQuoteQty']) * fee_factor
        free_balance, _ = get_asset_balance(client, quote_asset[1])
    if qty > free_balance:
        qty = free_balance
    order1 = create_market_order(client, symbol[1], side[1], qty)
    print(order1)

    symbol[2] = base_asset[2] + quote_asset[2]
    if side[2] == 'SELL':
        qty = float(order1['executedQty']) * fee_factor
        free_balance, _ = get_asset_balance(client, base_asset[2])
    elif side[2] == 'BUY':
        qty = float(order1['cummulativeQuoteQty']) * fee_factor
        free_balance, _ = get_asset_balance(client, quote_asset[2])
    if qty > free_balance:
        qty = free_balance
    order2 = create_market_order(client, symbol[2], side[2], qty)
    print(order2)

    order0_df = order_to_df(order0)
    order1_df = order_to_df(order1)
    order2_df = order_to_df(order2)

    return pd.concat([order0_df, order1_df, order2_df]).set_index('symbol')

In [ ]:
def calculate_arbitrage_gain(symbols, sides, prices_df, balance, fee):
    prices = [np.nan] * 3
    prices[0] = get_trunc_value(float(prices_df[prices_df.index == symbols[0]]['price'][0]), 0.00000001)
    prices[1] = get_trunc_value(float(prices_df[prices_df.index == symbols[1]]['price'][0]), 0.00000001)
    prices[2] = get_trunc_value(float(prices_df[prices_df.index == symbols[2]]['price'][0]), 0.00000001)

    if sides[0] == 'BUY':
        gain = balance / prices[0] * fee * prices[1] * fee / prices[2] * fee
    else:
        gain = balance * prices[0] * fee / prices[1] * fee * prices[2] * fee

    return gain / balance, prices

def get_arbitrage_gain(base_asset, quote_asset, end_quote_asset, prices_df, balance):
    arbitrage_gains = {}

    base_assets_sell = [quote_asset, base_asset, base_asset]
    quote_assets_sell = [end_quote_asset, end_quote_asset, quote_asset]
    symbols_sell = [base_assets_sell[0] + quote_assets_sell[0], base_assets_sell[1] + quote_assets_sell[1], base_assets_sell[2] + quote_assets_sell[2]]
    sides_sell = ['SELL', 'BUY', 'SELL']
    base_assets_buy = [base_asset, base_asset, quote_asset]
    quote_assets_buy = [quote_asset, end_quote_asset, end_quote_asset]
    symbols_buy = [base_assets_buy[0] + quote_assets_buy[0], base_assets_buy[1] + quote_assets_buy[1], base_assets_buy[2] + quote_assets_buy[2]]
    sides_buy = ['BUY', 'SELL', 'BUY']

    gain_sell, prices_sell = calculate_arbitrage_gain(symbols_sell, sides_sell, prices_df, balance, fee_factor)
    gain_buy, prices_buy = calculate_arbitrage_gain(symbols_buy, sides_buy, prices_df, balance, fee_factor)

    if gain_sell > gain_buy:
        arbitrage_gains = {'base_assets': base_assets_sell, 
                           'quote_assets': quote_assets_sell,
                           'sides': sides_sell, 
                           'prices': prices_sell,
                           'gain': gain_sell}
    else:
        arbitrage_gains = {'base_assets': base_assets_buy, 
                           'quote_assets': quote_assets_buy, 
                           'sides': sides_buy, 
                           'prices': prices_buy,
                           'gain': gain_buy}
    
    return arbitrage_gains


In [ ]:
def order_to_df(order):
    order_df = pd.DataFrame(order)[['symbol', 'orderId', 'transactTime', 'origQty', 'executedQty', 'cummulativeQuoteQty', 'status', 'type', 'side']]
    order_df['transactTime']  = pd.to_datetime(order_df['transactTime'], unit='ms')
    fills_df = pd.DataFrame(order['fills'])
    order_df['price'] = fills_df['price']
    order_df['qty'] = fills_df['qty']

    return order_df

In [ ]:
from binance_utils import get_all_lastest_price

#end_quote_assets = set(symbols_df[symbols_df['baseAsset'] == arbitrage_asset]['quoteAsset'])
end_quote_assets = ['DAI', 'BUSD', 'GBP', 'AUD', 'BRL', 'USDT', 'EUR', 'USDC', 'RUB', 'UST', 'ETH', 'BTC']

arbitrage_symbols = []
orders_df = pd.DataFrame()
arbitrage_opportunity_found = False
while not arbitrage_opportunity_found:
    #prices_df = get_all_lastest_price(client)
    for index, row in symbols_df[symbols_df['quoteAsset'] == arbitrage_asset].iterrows():
        prices_df = get_all_lastest_price(client)
        for end_quote_asset in end_quote_assets:
            # verify if all the pairs exist in symbols_df
            if ((len(symbols_df[(symbols_df['baseAsset'] == row['baseAsset']) & (symbols_df['quoteAsset'] == end_quote_asset)]) > 0) & 
                    (len(symbols_df[(symbols_df['baseAsset'] == row['quoteAsset']) & (symbols_df['quoteAsset'] == end_quote_asset)]) > 0)):
                arbitrage_gains = get_arbitrage_gain(row['baseAsset'], 
                                                row['quoteAsset'], 
                                                end_quote_asset, 
                                                prices_df, 
                                                initial_arbitrage_balance)

                if (arbitrage_gains['gain'] > gain_threshold and not arbitrage_opportunity_found):
                    arbitrage_symbols.append((arbitrage_gains['base_assets'],
                                              arbitrage_gains['quote_assets'],
                                              arbitrage_gains['sides'],
                                              arbitrage_gains['gain'], 
                                              row['cmcRank'],
                                              '{:0.8f}'.format(arbitrage_gains['prices'][0]), 
                                              '{:0.8f}'.format(arbitrage_gains['prices'][1]), 
                                              '{:0.8f}'.format(arbitrage_gains['prices'][2])))
                    
                    if (create_orders):
                        orders_df = execute_arbitrage_orders(arbitrage_gains['base_assets'],
                                                             arbitrage_gains['quote_assets'], 
                                                             arbitrage_gains['sides'], 
                                                             initial_arbitrage_balance)
                        #print(orders_df)

                    arbitrage_opportunity_found = True
                ###
                #print(arbitrage_gains)

    
    # to not to repeat the while loop uncomment next line
    #arbitrage_opportunity_found = True

arbitrage_symbols = pd.DataFrame(arbitrage_symbols, columns=['base_assets', 'quote_assets', 'sides', 'gain', 'cmc_rank', 'price0', 'price1', 'price2'])
arbitrage_symbols = arbitrage_symbols.sort_values(by='gain', ascending=False)

In [ ]:
arbitrage_symbols

In [ ]:
assets = [base_asset for base_assets in arbitrage_symbols['base_assets'] + arbitrage_symbols['quote_assets'] for base_asset in base_assets]
assets = set(assets)

In [ ]:
info = client.get_account()
assets_after_arbitrage_df = pd.DataFrame(info['balances'])

In [ ]:
assets_before_arbitrage_df[assets_before_arbitrage_df['asset'].isin(assets)]

In [ ]:
assets_after_arbitrage_df[assets_after_arbitrage_df['asset'].isin(assets)]

In [ ]:
orders_df